In [3]:
from scipy.io import wavfile
import torch
import sounddevice as sd
import pytorch_lightning as pl
from trainer import Iminegg
from my_iterable_dataset import MyIterableDataset
from im_in_egg_unet1_param import ImineggNet1
from torch.utils.data import DataLoader

In [4]:
ds_validate = MyIterableDataset(100, "./data/val", 22528 * 4)

c:\ML\iminegg\my_iterable_dataset.py:19: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  self.voices.append(torch.from_numpy(data).float() / 32768)


In [8]:
iminegg_net = ImineggNet1(32, True)
iminegg = Iminegg(iminegg_net)
validate_set = DataLoader(ds_validate, batch_size=20)
ckpt = "epoch=503_no_speach_split.ckpt" 
ckpt = "logs/defaultw128-bs20/version_8/checkpoints/epoch=875.ckpt"
ckpt = "logs/defaultw32-bs40/version_1/checkpoints/epoch=357.ckpt"
trainer = pl.Trainer(resume_from_checkpoint=ckpt, gpus=0)
trainer.test(model=iminegg, test_dataloaders=validate_set, ckpt_path=ckpt)


GPU available: True, used: False
TPU available: False, using: 0 TPU cores


1

In [14]:
samplerate, data = wavfile.read('./tests/source_17.wav')
data = (torch.from_numpy(data).float() / 32768)
sec1 = 11264
off = 0
length = (len(data) - off) // sec1 * sec1

inp = data[off:off + length]

In [12]:
d = next(iter(ds_validate))
inp = d['input']
print(d['type'])
device = torch.device("cpu")

2


In [15]:
sd.play(inp.cpu().numpy(), samplerate)

In [16]:
trainer.model.net.eval()
outp = trainer.model.net.calc(inp.reshape(1, 1, -1).to(device))
#outp = trainer.model.net.calc(outp.reshape(1, 1, -1).to(device))

In [17]:
sd.play(outp.cpu().detach().numpy().reshape(-1), samplerate)

In [766]:
export_outp = (outp.cpu() * 32768).short()
export_inp = (inp.cpu() * 32768).short()

wavfile.write('tests/filtered_20.wav', samplerate, export_outp.detach().numpy())
wavfile.write('tests/source_20.wav', samplerate, export_inp.detach().numpy())